In [47]:
import pandas as pd
import numpy as np
import random

In [48]:
def duplicar_random(file_path, n):
    lines = open(F"{file_path}").readlines()
    lista_archivos = []
    for i in range(n):
        random.shuffle(lines)
        with open(F"../scripts/instancias/random/random-{i}.txt", 'w') as f:
            f.writelines(lines)
        lista_archivos.append(F"../scripts/instancias/random/random-{i}.txt")
    with open(F"instancias/random/indice.txt", 'w') as f:
        for j in range(n):
            print(lista_archivos[j], file=f, end=' ')

In [49]:
duplicar_random("../data/corpus", 100)